## Installation

In [1]:
!pip install -q gradio huggingface_hub

     |████████████████████████████████| 11.6 MB 3.8 MB/s 
     |████████████████████████████████| 163 kB 50.5 MB/s 
     |████████████████████████████████| 106 kB 48.3 MB/s 
     |████████████████████████████████| 213 kB 46.4 MB/s 
     |████████████████████████████████| 55 kB 803 kB/s 
     |████████████████████████████████| 84 kB 579 kB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 2.3 MB 3.0 MB/s 
     |████████████████████████████████| 56 kB 2.1 MB/s 
     |████████████████████████████████| 84 kB 1.1 MB/s 
     |████████████████████████████████| 272 kB 39.3 MB/s 
     |████████████████████████████████| 63 kB 760 kB/s 
     |████████████████████████████████| 80 kB 2.8 MB/s 
     |████████████████████████████████| 68 kB 2.8 MB/s 
     |████████████████████████████████| 46 kB 2.0 MB/s 
     |████████████████████████████████| 593 kB 54.8 MB/s 
     |████████████████████████████████| 4.1 MB 40.7 MB/s 
     |███████████████████████████

## Fetch a model from the HF Hub

In [2]:
from huggingface_hub import from_pretrained_keras


model_ckpt = "chansung/segmentation-training-pipeline@v1667662600"

MODEL = from_pretrained_keras(model_ckpt)

config.json not found in HuggingFace Hub


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/471k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/63.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.9k [00:00<?, ?B/s]

## Fetch a test image

In [3]:
!wget https://i.ibb.co/p44PvmN/test-image1.png -O test-image1.png
!wget https://i.ibb.co/brJnX3m/test-image2.png -O test-image2.png

--2022-11-05 17:02:38--  https://i.ibb.co/p44PvmN/test-image1.png
Resolving i.ibb.co (i.ibb.co)... 104.243.38.202, 104.243.38.177
Connecting to i.ibb.co (i.ibb.co)|104.243.38.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1049346 (1.0M) [image/png]
Saving to: ‘test-image1.png’

test-image1.png     100%[===================>]   1.00M  1.13MB/s    in 0.9s    

2022-11-05 17:02:39 (1.13 MB/s) - ‘test-image1.png’ saved [1049346/1049346]

--2022-11-05 17:02:40--  https://i.ibb.co/brJnX3m/test-image2.png
Resolving i.ibb.co (i.ibb.co)... 104.243.38.202, 104.243.38.177
Connecting to i.ibb.co (i.ibb.co)|104.243.38.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 954734 (932K) [image/png]
Saving to: ‘test-image2.png’

test-image2.png     100%[===================>] 932.36K   992KB/s    in 0.9s    

2022-11-05 17:02:41 (992 KB/s) - ‘test-image2.png’ saved [954734/954734]



## Utilities

In [27]:
from PIL import Image 
import numpy as np 
import tensorflow as tf 


RESOLTUION = 128
# MODEL = tf.keras.models.load_model(model_path)

def preprocess_input(image: Image) -> tf.Tensor:
    image = np.array(image)
    image = tf.convert_to_tensor(image)

    image = tf.image.resize(image, (RESOLTUION, RESOLTUION))
    image = image / 255

    return tf.expand_dims(image, 0)


# The below utilities (sidewalk_palette(), get_seg_overlay()) are from:
# https://github.com/deep-diver/semantic-segmentation-ml-pipeline/blob/main/notebooks/inference_from_SavedModel.ipynb

def pets_palette():
    """Pets palette that maps each class to RGB values."""
    return [
        [  0,  10, 146],
        [ 38,   0,  44],
        [255, 232,   0],
    ]


def get_seg_overlay(image, seg):
    color_seg = np.zeros(
        (seg.shape[0], seg.shape[1], 3), dtype=np.uint8
    )  # height, width, 3
    palette = np.array(pets_palette())
    
    for label, color in enumerate(palette):
        color_seg[seg == label, :] = color

    # Show image + mask
    img = np.array(image) * 0.5 + color_seg * 0.5

    img *= 255
    img = np.clip(img, 0, 255)
    img = img.astype(np.uint8)
    return img


def run_model(image: Image) -> tf.Tensor:
    preprocessed_image = preprocess_input(image)
    prediction = MODEL.predict(preprocessed_image)

    seg_mask = tf.math.argmax(prediction, -1)
    seg_mask = tf.squeeze(seg_mask)
    return seg_mask


def get_predictions(image: Image):
    predicted_segmentation_mask = run_model(image)
    preprocessed_image = preprocess_input(image)
    preprocessed_image = tf.squeeze(preprocessed_image, 0)
    
    pred_img = get_seg_overlay(preprocessed_image.numpy(), predicted_segmentation_mask.numpy())
    return Image.fromarray(pred_img)

## Gradio demo

In [28]:
import gradio as gr

title = "Simple demo for a semantic segmentation model trained on the Sidewalks dataset."

description = """

Note that the outputs obtained in this demo won't be state-of-the-art. The underlying project has a different objective focusing more on the ops side of
deploying a semantic segmentation model. For more details, check out the repository: https://github.com/deep-diver/semantic-segmentation-ml-pipeline/.

"""

demo = gr.Interface(
    get_predictions,
    gr.inputs.Image(type="pil"),
    "pil",
    allow_flagging="never",
    title=title,
    description=description,
    examples=[["test-image1.png"], ["test-image2.png"]]
)

demo.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().

Using Embedded Colab Mode (NEW). If you have issues, please use share=True and file an issue at https://github.com/gradio-app/gradio/
Note: opening the browser inspector may crash Embedded Colab Mode.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 60ms/step
Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7fdb791bf0d0>, 'http://127.0.0.1:7860/', None)

To add more examples, do - `examples=[["test-image.jpg"], ["image2.jpg"], ["sample.png"], ...]`.